In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession 
    .builder 
    .appName("Dibimbing Spark-Kafka") 
    .config("spark.streaming.stopGracefullyOnShutdown", True) 
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.2')
    .config("spark.sql.shuffle.partitions", 4)
    .master("local[*]") 
    .getOrCreate()
)

spark

In [2]:
# Read input data

df_raw = spark.readStream.format("socket").option("host","localhost").option("port", "9999").load()

In [3]:
df_raw.printSchema()

root
 |-- value: string (nullable = true)



In [4]:
# Split the line into words
from pyspark.sql.functions import split

df_words = df_raw.withColumn("words", split("value", " "))

In [5]:

# Explode the list of words
from pyspark.sql.functions import explode

df_explode = df_words.withColumn("word", explode("words")).drop("value", "words")

In [6]:
# Aggregate the words to generate count
from pyspark.sql.functions import count, lit

df_agg = df_explode.groupBy("word").agg(count(lit(1)).alias("cnt"))

In [7]:
# Write the output to console streaming

streaming_obj = df_agg.writeStream.format("console").outputMode("append").start()

AnalysisException: Append output mode not supported when there are streaming aggregations on streaming DataFrames/DataSets without watermark;
Aggregate [word#6], [word#6, count(1) AS cnt#13L]
+- Project [word#6]
   +- Project [value#0, words#2, word#6]
      +- Generate explode(words#2), false, [word#6]
         +- Project [value#0, split(value#0,  , -1) AS words#2]
            +- StreamingRelationV2 org.apache.spark.sql.execution.streaming.sources.TextSocketSourceProvider@661a5584, socket, org.apache.spark.sql.execution.streaming.sources.TextSocketTable@3a7fcda1, [host=localhost, port=9999], [value#0]


In [8]:
# streaming_obj.stop()
# spark.stop()